In [5]:
from semantic_kernel import Kernel
from semantic_kernel.functions import kernel_function, KernelArguments
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from utils import *
import logging

In [6]:
# 0. Enable logging
# NOTE: This is all that is required to enable logging.
# Set the desired level to INFO, DEBUG, etc.
logging.basicConfig(level=logging.INFO)

In [7]:
service = AzureChatCompletion(
    service_id="sk_agent_service", api_key=azure_openai_api_key,deployment_name=azure_openai_deployment,
    api_version=azure_openai_api_version,endpoint=azure_openai_endpoint

)

settings = AzureChatPromptExecutionSettings(service_id="sk_agent_service")
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
thread = ChatHistoryAgentThread()

# Create the agent with the kernel, thread and plugin

prompt = """You MUST use the search_retrieval function for ALL queries. Do not paraphrase more than the result. Never generate answers from prior knowledge.
    When you receive search results:
    1. If the source is "Azure AI Search", these are knowledge base entries. Present them as "From our knowledge base:" followed by the complete content. Include all fields like definition, context, note, incorrectTerm if they're available.
    2. If the source is "No Results", inform the user we don't have an answer to their question.
    
    Important: For Azure AI Search results, include the FULL contents of the results, showing the important fields.
    
    Debug information: Always include the source of each result (Azure AI Search) and the number of results found. If you received results but are not displaying them, explain why.
    
    Always show ALL results you receive - do not filter them based on score thresholds.
    Summarize the response without adding any new information. Provide the page number of the document if available."""

sk_agent = ChatCompletionAgent(
    service=service,
    name="sk_agent",
    instructions=prompt,
    plugins=[SearchRetrievalPlugin()],
    arguments=KernelArguments(settings=settings),
)

In [ ]:
async def chat_with_agent():
    global thread # access the global thread variable
    while True:
        user_input = input("User prompt: ")

        if user_input.lower() in ["quit", "exit", "q", "end"]:
            print("\n\nExiting chat..")
            print("Good bye, please let me know if you need further help.")
            break

        try:
            print(f"User: {user_input}")
            async for response in sk_agent.invoke(messages=user_input, thread=thread):
                print(f"{response.name}: {response}")
                thread = response.thread
                #print(f"# {response.name}: {response}")
            print("\n")  # New line after complete response
            
        except Exception as e:
            print(f"An error occurred: {e}")

# Run the async function
await chat_with_agent()

User: my name is Charles
sk_agent: Nice to meet you, Charles! How can I assist you today?


User: tell me about expanding play 3
sk_agent: From our knowledge base:

1. **Accelerating Sustainability with AI: Play 3**  
   Play 3 focuses on minimizing resource use, expanding access to carbon-free electricity, and supporting local communities. It acknowledges that AI operations require resources such as energy and water. To align AI operations with global sustainability goals and achieve a net zero future, Play Three emphasizes minimizing the sustainability impact of AI by not only reducing resource use in data centers but also supporting the decarbonization of local energy grids. This is crucial because even though data centers currently account for a small percentage of global electricity consumption, the projected growth is concentrated in specific regions like Ireland and Denmark, which might see an increase in electricity demand due to data centers. [Source: Azure AI Search, Page 20]